In [1]:
import pandas as pd

# Load the dataset (update 'file_path' with the actual file path)
file_path = 'API.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

# Count the occurrences of each API
top_apis = data['API Code'].value_counts().head(4).index.tolist()

# Filter and save each top API to a separate CSV file
for api in top_apis:
    api_data = data[data['API Code'] == api]
    file_name = f"top_{api}_usage.csv"
    api_data.to_csv(file_name, index=False)
    print(f"Saved {file_name}")


Saved top_A9_usage.csv
Saved top_A2_usage.csv
Saved top_A7_usage.csv
Saved top_A4_usage.csv


In [2]:
pip install pandas numpy tensorflow scikit-learn matplotlib


# New Section

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Load the dataset (replace with your file path)
data = pd.read_csv("top_A2_usage.csv")  # Replace with actual file
data['Time of Call'] = pd.to_datetime(data['Time of Call'],dayfirst=True)
data.set_index('Time of Call', inplace=True)

# Aggregate API calls by time (e.g., hourly)
time_series = data.resample('H').count()['API Code']

# Normalize the data
scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

# Create supervised learning dataset
def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24  # Use the last 24 hours to predict the next step
X, y = create_supervised(time_series_scaled, lag)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-41-5e9ecd5ef73d>:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [43]:
from tensorflow.keras.layers import GRU

def build_gru(input_shape):
    model = Sequential()
    model.add(GRU(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [44]:
def build_mlp(input_shape):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_shape[0],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [45]:
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D

def build_fcn(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [46]:
from tensorflow.keras.layers import Add, BatchNormalization

def build_resnet(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, padding='same', activation='relu')(input_layer)
    x = BatchNormalization()(x)
    residual = x

    for _ in range(3):  # Add 3 residual blocks
        x = Conv1D(64, kernel_size=3, padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = Add()([x, residual])
        residual = x

    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [48]:
import tensorflow as tf

In [49]:
from math import sqrt

def train_and_evaluate(model_builder, X_train, y_train, X_test, y_test, model_name):
    model = model_builder(X_train.shape[1:])
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return model, rmse

# Train and evaluate all models
models = {
    "LSTM": build_lstm,
    "GRU": build_gru,
    "MLP": build_mlp,
    "FCN": build_fcn,
    "ResNet": build_resnet,
}

results = {}
for name, builder in models.items():
    model, rmse = train_and_evaluate(builder, X_train, y_train, X_test, y_test, name)
    results[name] = rmse

# Print results
print("Model Performance:")
for model_name, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model_name}: RMSE = {rmse}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
LSTM RMSE: 0.19032063865857665


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
GRU RMSE: 0.19167080856463697


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MLP RMSE: 0.21118306462561762


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
FCN RMSE: 0.19068726677733489
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
ResNet RMSE: 0.20700977265612594
Model Performance:
LSTM: RMSE = 0.19032063865857665
FCN: RMSE = 0.19068726677733489
GRU: RMSE = 0.19167080856463697
ResNet: RMSE = 0.20700977265612594
MLP: RMSE = 0.21118306462561762


In [71]:
# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name}")

# Retrain the best model on the entire dataset
best_model = models[best_model_name](X_train.shape[1:])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Start with the last observed input
current_input = X_test[-1].reshape(1, -1, 1)

# Define time horizons
horizons = [1, 24, 24 * 7, 24 * 30]  # Next hour, day, week, year

# Sequential prediction for specific horizons
future_predictions = []
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):
        prediction = best_model.predict(current_input, verbose=0)
        future_predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)



future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (Day 1)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for Days 1-7 (Week)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for Days 1-30 (Month)

# Print results
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: LSTM


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 14
Total Calls Next Week (Days 1-7): 101
Total Calls Next Month (Days 1-30): 434


In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Load the dataset (replace with your file path)
data = pd.read_csv("top_A4_usage.csv")  # Replace with actual file
data['Time of Call'] = pd.to_datetime(data['Time of Call'],dayfirst=True)
data.set_index('Time of Call', inplace=True)

# Aggregate API calls by time (e.g., hourly)
time_series = data.resample('H').count()['API Code']

# Normalize the data
scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

# Create supervised learning dataset
def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24  # Use the last 24 hours to predict the next step
X, y = create_supervised(time_series_scaled, lag)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-72-34265028b870>:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [73]:
from tensorflow.keras.layers import LSTM, Conv1D, GlobalAveragePooling1D, Dropout, concatenate

def build_lstm_fcn(input_shape):
    # LSTM branch
    input_layer = tf.keras.Input(shape=input_shape)
    lstm_out = LSTM(64)(input_layer)

    # FCN branch
    conv1 = Conv1D(128, kernel_size=8, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(256, kernel_size=5, activation='relu', padding='same')(conv1)
    conv3 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv2)
    gap = GlobalAveragePooling1D()(conv3)

    # Merge branches
    merged = concatenate([lstm_out, gap])
    output_layer = Dense(1)(merged)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [74]:
def build_gru_fcn(input_shape):
    # GRU branch
    input_layer = tf.keras.Input(shape=input_shape)
    gru_out = GRU(64)(input_layer)

    # FCN branch
    conv1 = Conv1D(128, kernel_size=8, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(256, kernel_size=5, activation='relu', padding='same')(conv1)
    conv3 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv2)
    gap = GlobalAveragePooling1D()(conv3)

    # Merge branches
    merged = concatenate([gru_out, gap])
    output_layer = Dense(1)(merged)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [79]:
pip install PyWavelets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 78.0 MB/s eta 0:00:00


In [80]:

import pywt

def wavelet_decomposition(data, wavelet='db1', level=3):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    reconstructed = pywt.waverec(coeffs, wavelet)
    return np.array(coeffs).T  # Transpose for model input

# Example model
def build_mwdn(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
    x = LSTM(64, return_sequences=True)(x)
    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [76]:
from tensorflow.keras.layers import LayerNormalization, Activation

def build_tcn(input_shape, nb_filters=64, kernel_size=3, nb_stacks=1, dilation_rates=[1, 2, 4, 8]):
    input_layer = tf.keras.Input(shape=input_shape)
    x = input_layer

    for _ in range(nb_stacks):
        for dilation_rate in dilation_rates:
            x = Conv1D(nb_filters, kernel_size, padding="causal", dilation_rate=dilation_rate)(x)
            x = LayerNormalization()(x)
            x = Activation('relu')(x)

    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [77]:
def build_mlstm_fcn(input_shape):
    # LSTM branch
    input_layer = tf.keras.Input(shape=input_shape)
    lstm_out = LSTM(64, return_sequences=True)(input_layer)
    lstm_out = LSTM(32)(lstm_out)

    # FCN branch
    conv1 = Conv1D(128, kernel_size=8, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(256, kernel_size=5, activation='relu', padding='same')(conv1)
    conv3 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv2)
    gap = GlobalAveragePooling1D()(conv3)

    # Merge branches
    merged = concatenate([lstm_out, gap])
    output_layer = Dense(1)(merged)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [81]:
from math import sqrt

def train_and_evaluate(model_builder, X_train, y_train, X_test, y_test, model_name):
    model = model_builder(X_train.shape[1:])
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return model, rmse

# Train and evaluate all models
models = {
    "LSTM_FCN": build_lstm_fcn,
    "GRU-FCN": build_gru_fcn,
    "mWDN": build_mwdn,
    "TCN ": build_tcn,
    "MLSTM-FCN": build_mlstm_fcn,
}

results = {}
for name, builder in models.items():
    model, rmse = train_and_evaluate(builder, X_train, y_train, X_test, y_test, name)
    results[name] = rmse

# Print results
print("Model Performance:")
for model_name, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model_name}: RMSE = {rmse}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
LSTM_FCN RMSE: 0.15139491339634664
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
GRU-FCN RMSE: 0.1538991454327839
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
mWDN RMSE: 0.1508137151764486
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
TCN  RMSE: 0.15982444656313743
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
MLSTM-FCN RMSE: 0.15527828173544408
Model Performance:
mWDN: RMSE = 0.1508137151764486
LSTM_FCN: RMSE = 0.15139491339634664
GRU-FCN: RMSE = 0.1538991454327839
MLSTM-FCN: RMSE = 0.15527828173544408
TCN : RMSE = 0.15982444656313743


In [82]:
# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name}")

# Retrain the best model on the entire dataset
best_model = models[best_model_name](X_train.shape[1:])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Start with the last observed input
current_input = X_test[-1].reshape(1, -1, 1)

# Define time horizons
horizons = [1, 24, 24 * 7, 24 * 30]  # Next hour, day, week, year

# Sequential prediction for specific horizons
future_predictions = []
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):
        prediction = best_model.predict(current_input, verbose=0)
        future_predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)



future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (Day 1)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for Days 1-7 (Week)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for Days 1-30 (Month)

# Print results
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: mWDN
Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 17
Total Calls Next Week (Days 1-7): 111
Total Calls Next Month (Days 1-30): 472


In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Load the dataset (replace with your file path)
data = pd.read_csv("top_A7_usage.csv")  # Replace with actual file
data['Time of Call'] = pd.to_datetime(data['Time of Call'],dayfirst=True)
data.set_index('Time of Call', inplace=True)

# Aggregate API calls by time (e.g., hourly)
time_series = data.resample('H').count()['API Code']

# Normalize the data
scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

# Create supervised learning dataset
def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24  # Use the last 24 hours to predict the next step
X, y = create_supervised(time_series_scaled, lag)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-83-6e7d9c243c4b>:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [84]:
import tensorflow as tf
from tensorflow.keras import layers

def build_inceptiontime(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    # First branch: 1x1 convolution
    branch1 = layers.Conv1D(32, 1, padding='same', activation='relu')(input_layer)

    # Second branch: 3x3 convolution
    branch2 = layers.Conv1D(32, 3, padding='same', activation='relu')(input_layer)

    # Third branch: 5x5 convolution
    branch3 = layers.Conv1D(32, 5, padding='same', activation='relu')(input_layer)

    # Concatenate all branches
    concatenated = layers.concatenate([branch1, branch2, branch3])

    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling1D()(concatenated)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)  # Change to 'Dense(num_classes, activation="softmax")' for classification

    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [86]:
def build_xceptiontime(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    # Xception block
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(input_layer)
    x = layers.SeparableConv1D(64, 3, padding='same', activation='relu')(x)
    x = layers.SeparableConv1D(128, 3, padding='same', activation='relu')(x)

    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)  # For regression, change for classification

    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [87]:
def build_rescnn(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    # First convolutional block with residual connections
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(input_layer)
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(x)
    residual = layers.Conv1D(32, 1, padding='same')(input_layer)  # Residual connection
    x = layers.add([x, residual])

    # Second convolutional block
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    residual = layers.Conv1D(64, 1, padding='same')(x)  # Residual connection
    x = layers.add([x, residual])

    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)  # For regression, change for classification

    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [146]:

def build_xcm(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = layers.Conv1D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [ ]:
from tensorflow.keras import layers, models

def build_tst(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [147]:
from math import sqrt

def train_and_evaluate(model_builder, X_train, y_train, X_test, y_test, model_name):
    model = model_builder(X_train.shape[1:])
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return model, rmse

# Train and evaluate all models
models = {
    "InceptionTime": build_inceptiontime,
    "XceptionTime": build_xceptiontime,
    "ResCNN": build_rescnn,
    "TST": build_tst,
    "XCM": build_xcm,

}

results = {}
for name, builder in models.items():
    model, rmse = train_and_evaluate(builder, X_train, y_train, X_test, y_test, name)
    results[name] = rmse

# Print results
print("Model Performance:")
for model_name, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model_name}: RMSE = {rmse}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
InceptionTime RMSE: 0.1583996205144647
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
XceptionTime RMSE: 0.1581217418184169
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ResCNN RMSE: 0.15848909298692937
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
XCM RMSE: 0.1581608162193207
Model Performance:
XceptionTime: RMSE = 0.1581217418184169
XCM: RMSE = 0.1581608162193207
InceptionTime: RMSE = 0.1583996205144647
ResCNN: RMSE = 0.15848909298692937


In [148]:
# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name}")

# Retrain the best model on the entire dataset
best_model = models[best_model_name](X_train.shape[1:])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Start with the last observed input
current_input = X_test[-1].reshape(1, -1, 1)

# Define time horizons
horizons = [1, 24, 24 * 7, 24 * 30]  # Next hour, day, week, year

# Sequential prediction for specific horizons
future_predictions = []
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):
        prediction = best_model.predict(current_input, verbose=0)
        future_predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)



future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (Day 1)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for Days 1-7 (Week)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for Days 1-30 (Month)

# Print results
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: XceptionTime
Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 17
Total Calls Next Week (Days 1-7): 120
Total Calls Next Month (Days 1-30): 512


In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Load the dataset (replace with your file path)
data = pd.read_csv("top_A9_usage.csv")  # Replace with actual file
data['Time of Call'] = pd.to_datetime(data['Time of Call'],dayfirst=True)
data.set_index('Time of Call', inplace=True)

# Aggregate API calls by time (e.g., hourly)
time_series = data.resample('H').count()['API Code']

# Normalize the data
scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

# Create supervised learning dataset
def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24  # Use the last 24 hours to predict the next step
X, y = create_supervised(time_series_scaled, lag)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-104-037b1aba7044>:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [150]:
from tensorflow.keras import layers, models

def build_tabtransformer(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



In [151]:
from tensorflow.keras import layers, models

def build_tsit(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = layers.LayerNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



In [152]:
from tensorflow.keras import layers, models

def build_gmlp(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



In [153]:
from tensorflow.keras import layers, models

def build_tsperceiver(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [154]:
def build_gated_tabtransformer(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [155]:
from tensorflow.keras import layers, models

def build_tssequencerplus(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.LSTM(128, return_sequences=True)(input_layer)
    x = layers.LSTM(64)(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



In [156]:
def build_patchtst(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(x, x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [162]:
from math import sqrt

def train_and_evaluate(model_builder, X_train, y_train, X_test, y_test, model_name):
    model = model_builder(X_train.shape[1:])
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return model, rmse

# Train and evaluate all models
#


models = {


     "PatchTST": build_patchtst,
    "TabTransformer": build_tabtransformer,
    "TSPerceiver": build_tsperceiver,
    "GatedTabTransformer": build_gated_tabtransformer,
    "TSSequencerPlus": build_tssequencerplus
}

results = {}
for name, builder in models.items():
    model, rmse = train_and_evaluate(builder, X_train, y_train, X_test, y_test, name)
    results[name] = rmse

# Print results
print("Model Performance:")
for model_name, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model_name}: RMSE = {rmse}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
PatchTST RMSE: 0.15831527391998212
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
TabTransformer RMSE: 0.16121663081541726
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
TSPerceiver RMSE: 0.1585658798221713
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
GatedTabTransformer RMSE: 0.15896102280398955
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
TSSequencerPlus RMSE: 0.15837739809113877
Model Performance:
PatchTST: RMSE = 0.15831527391998212
TSSequencerPlus: RMSE = 0.15837739809113877
TSPerceiver: RMSE = 0.1585658798221713
GatedTabTransformer: RMSE = 0.15896102280398955
TabTransformer: RMSE = 0.16121663081541726


In [163]:
# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name}")

# Retrain the best model on the entire dataset
best_model = models[best_model_name](X_train.shape[1:])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Start with the last observed input
current_input = X_test[-1].reshape(1, -1, 1)

# Define time horizons
horizons = [1, 24, 24 * 7, 24 * 30]  # Next hour, day, week, year

# Sequential prediction for specific horizons
future_predictions = []
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):
        prediction = best_model.predict(current_input, verbose=0)
        future_predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)



future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (Day 1)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for Days 1-7 (Week)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for Days 1-30 (Month)

# Print results
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: PatchTST
Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 16
Total Calls Next Week (Days 1-7): 108
Total Calls Next Month (Days 1-30): 464
